In [1]:
import pandas as pd

data = r"D:\Laboral\World Bank\Data-Portal-Brief-Generator\Data"
data_raw = fr"{data}\Data_Raw"
data_processed = fr"{data}\Data_Processed" 

complete_series_wmetadata = pd.read_stata(fr"{data_processed}\complete_series_wmetadata.dta")
names = pd.read_excel(fr"{data_raw}\Country codes & metadata\metadata.xlsx")
coded_names = names.code.str.replace(".","_").unique()
variables = pd.Series(complete_series_wmetadata.code.unique())
assert variables.isin(coded_names).all(), f"There are indicators without metadata: {variables[-variables.isin(coded_names)]}"

C:\Users\ofici\AppData\Local\Temp\ipykernel_3720\4217078062.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  coded_names = names.code.str.replace(".","_").unique()


AssertionError: There are indicators without metadata: 36             MNCH_DIARCARE
89     netenrolment_lowersec
90         netenrolment_prim
91     netenrolment_uppersec
102       unicef_stillbirths
107              HD.HCI.EYRS
108           HD.HCI.EYRS.FE
109           HD.HCI.EYRS.MA
110              HD.HCI.STNT
111           HD.HCI.STNT.FE
112           HD.HCI.STNT.MA
113              HD.HCI.HLOS
114           HD.HCI.HLOS.FE
115           HD.HCI.HLOS.MA
116              HD.HCI.OVRL
117           HD.HCI.OVRL.FE
118        HD.HCI.OVRL.LB.FE
119        HD.HCI.OVRL.UB.FE
120           HD.HCI.OVRL.LB
121           HD.HCI.OVRL.MA
122        HD.HCI.OVRL.LB.MA
123        HD.HCI.OVRL.UB.MA
124           HD.HCI.OVRL.UB
125              HD.HCI.LAYS
126           HD.HCI.LAYS.FE
127           HD.HCI.LAYS.MA
128              HD.HCI.MORT
129           HD.HCI.MORT.FE
130           HD.HCI.MORT.MA
131              HD.HCI.AMRT
132           HD.HCI.AMRT.FE
133           HD.HCI.AMRT.MA
dtype: object

In [1]:
api_access = __import__('01_API_access')
# api_access.query_indicator()
indics = api_access.get_list_of_unicef_indicators()

C:\Users\ofici\AppData\Roaming\Python\Python39\site-packages\pandasdmx\remote.py:11: RuntimeWarning: optional dependency requests_cache is not installed; cache options to Session() have no effect
  warn(


In [2]:
indicators = api_access.get_list_of_unicef_indicators()
cl_ref_areas = api_access.get_county_codes()


In [ ]:
selected_ind_data = indicators.loc[selected_ind]
selected_ind_code = selected_ind_data.name
selected_ind_name = selected_ind_data['name']
selected_ind_parent = selected_ind_data['parent']
            
# Parents in WASH are not correct
#   FIXME: if this raises errors, they must have fixed the WASH dataset. Check previous fixme...
if selected_ind in wash_indicators_house:
    selected_ind_parent = 'WASH_HOUSEHOLDS'
elif selected_ind in wash_indicators_school:
    selected_ind_parent = 'WASH_SCHOOLS'
elif selected_ind in wash_indicators_health:
    selected_ind_parent = 'WASH_HEALTHCARE_FACILITY'
elif selected_ind == 'ECD_CHLD_36-59M_LMPSL':
    selected_ind_parent = 'ECD'
elif selected_ind == 'C040202':
    selected_ind_parent == 'SDG_PROG_ASSESSMENT'
    
df = api_access.query_indicator(selected_ind_code, selected_ind_parent)
datasets[selected_ind_code] = api_access.format_dataframe(df, cl_ref_areas)


In [87]:
indics[indics.index == 'NT_BF_EXBF']

,name,parent
CL_UNICEF_INDICATOR,,
NT_BF_EXBF,Exclusive breastfeeding (0-5 months),NUTRITION


In [104]:
df = api_access.query_indicator('NT_CF_MMF', 'NUTRITION')
# api_access.drops_irrelevant_index_levels(df)
# api_access.format_dataframe(df, cl_ref_areas)

In [105]:
df

value
REF_AREA INDICATOR SEX AGE    WEALTH_QUINTILE RESIDENCE MATERNAL_EDU_LVL HEAD_OF_HOUSE TIME_PERIOD      
AFG      NT_CF_MMF F   M6T23  _T              _T        _T               _T            2015         48.7
                   M   M6T23  _T              _T        _T               _T            2015         49.1
                   _T  M10T11 _T              _T        _T               _T            2015         45.7
                       M12T13 _T              _T        _T               _T            2015         51.4
                       M12T15 _T              _T        _T               _T            2015         51.1
...                                                                                                  ...
ZWE      NT_CF_MMF _T  M6T7   _T              _T        _T               _T            2019         69.9
                       M8T9   _T              _T        _T               _T            2010         48.0
                                                                                       2014         59.2
                                                                                       2015         43.3
                                                                                       2019         73.4

[5646 rows x 1 columns]

In [102]:
indicator_0_to_5_years = [
    'NT_ANT_HAZ_NE2', 'NT_ANT_WHZ_NE2', 
    'NT_ANT_WHZ_NE3', 'NT_ANT_WHZ_PO2',
    'NT_BF_EXBF'
    ]
indicator_6_to_23_months =[
    'NT_CF_MMF'
]
indicator_15_to_49_years = [
    'MNCH_ANC1', 'MNCH_ANC4',
    'MNCH_DEMAND_FP', 'MNCH_PNCMOM',
    'MNCH_SAB', 'MNCH_ITNPREG', 
    ]
indicator_18_to_29_years = [
    'PT_M_18-29_SX-V_AGE-18',
    'PT_F_18-29_SX-V_AGE-18',
]
indicator_15_to_24_years = [
    'HVA_PREV_KNOW', 'HVA_PREV_KNOW_TEST', 
    'HVA_PREV_CNDM_MULT'
]

# Rename if the columns are wrong
df = df.rename(index={'COUNTRY': 'REF_AREA'})

indicador = df.index.get_level_values('INDICATOR').unique().values[0]

indexes = df.index.names

irrelevant_indexes = [
    idx for idx in indexes if idx not in ['REF_AREA', 'INDICATOR', 'TIME_PERIOD', 'SEX', 'DATA_SOURCE']]

for idx in irrelevant_indexes:
    totals_mask = df.index.get_level_values(idx) == '_T'
    if totals_mask.sum() > 0:
        df = df[totals_mask].droplevel(idx)
    elif idx == 'AGE':
        if indicador in indicator_0_to_5_years:
            df = df[df.index.get_level_values('AGE').isin(['Y0T4','M0T5'])]
        elif indicador in indicator_15_to_49_years:
            df = df[df.index.get_level_values('AGE') == 'Y15T49']
        elif indicador in indicator_18_to_29_years:
            df = df[df.index.get_level_values('AGE') == 'Y18T29']
        elif indicador in indicator_15_to_24_years:
            df = df[df.index.get_level_values('AGE') == 'Y15T24']
        else:
            print(f"Posible problema con {idx}")
        df = df.droplevel(idx)

    else:
        print(f"Posible problema con {idx}")
        df = df.droplevel(idx)

In [103]:
df

value
REF_AREA INDICATOR  SEX TIME_PERIOD      
AFG      NT_BF_EXBF F   2015         43.2
                    M   2015         43.0
                    _T  2015         43.1
                        2018         57.5
AGO      NT_BF_EXBF F   2015         40.1
...                                   ...
ZWE      NT_BF_EXBF M   2019         42.1
                    _T  2010         31.3
                        2014         40.3
                        2015         47.1
                        2019         41.9

[747 rows x 1 columns]

In [23]:
df

value
COUNTRY INDICATOR TIME_PERIOD          
AFG     C040202   2020             None
ALB     C040202   2010         74.16339
                  2011          78.8836
                  2012         84.94154
                  2013         86.70171
...                                 ...
ABW     C040202   2014         99.98893
PRI     C040202   2010         70.61019
                  2011         97.21883
                  2012         85.48009
                  2013         85.02325

[1300 rows x 1 columns]

In [18]:
api_access.drops_irrelevant_index_levels(
    api_access.query_indicator('C040202', 'SDG_PROG_ASSESSMENT')
)

AssertionError: There are duplicated rows.

In [13]:
api_access.get_indicators_description()


,name,parent
CL_UNICEF_INDICATOR,,
CME,Child mortality,CL_UNICEF_INDICATOR
CME_ARR_10T19,Annual Rate of Reduction in Mortality Rate Age...,CME
CME_ARR_SBR,Annual rate of reduction in stillbirth rate (o...,CME
CME_ARR_U5MR,Annual rate of reduction in under-five mortali...,CME
CME_MRM0,Neonatal mortality rate,CME
...,...,...
FD_NEVER_ATTENDED,Never attended school,FD
FD_FOUNDATIONAL_LEARNING,Foundational learning skills,FD
FD_SEVER_PHY_PUNISH,Severe physical punishment,FD


In [16]:
import pandasdmx as sdmx

# Get dataset structure from UNESCO
unicef = sdmx.Request('UNICEF') # Queries the UNICEF database
params = dict(startPeriod='2010') # FIXME: ¿Desde que año hay que descargar los datos?

# Only immunisation
exr_msg = unicef.dataflow('WASH_HOUSEHOLDS')
exr_flow = exr_msg.dataflow['WASH_HOUSEHOLDS']
dsd = exr_flow.structure
dsd.dimensions.components

# Gets the dimensions REF_AREA of the dataflow -> Countries and regions.
codelist = dsd.dimensions.get('INDICATOR').local_representation.enumerated
cl_indicators = sdmx.to_pandas(codelist)


In [17]:
cl_indicators

,name,parent
CL_UNICEF_INDICATOR,,
CME,Child mortality,CL_UNICEF_INDICATOR
CME_ARR_10T19,Annual Rate of Reduction in Mortality Rate Age...,CME
CME_ARR_SBR,Annual rate of reduction in stillbirth rate (o...,CME
CME_ARR_U5MR,Annual rate of reduction in under-five mortali...,CME
CME_MRM0,Neonatal mortality rate,CME
...,...,...
FD_NEVER_ATTENDED,Never attended school,FD
FD_FOUNDATIONAL_LEARNING,Foundational learning skills,FD
FD_SEVER_PHY_PUNISH,Severe physical punishment,FD


In [10]:
indics[indics.index == 'C040202']

,name,parent
CL_UNICEF_INDICATOR,,


In [3]:
complete_series_wmetadata[complete_series_wmetadata.code == "CME_SBR"]

,wbcode,year,gender,code,value
